# 0x0B Somebody is naughty

This guy is not verifying my transaction and creating two new of his own to do a reorg.  
_So it is time for a reorg..._

In [31]:
import math
import time
import json
from tqdm.notebook import tqdm
from sqlitedict import SqliteDict
from zimcoin_miner import ZimcoinMiner
from blocks import Block
from blocks import mine_block
from persistence import Persistence, dict_to_block
from node import Node
from connections import run_server, remote_connection
from blockchain_state import BlockchainState, verify_reorg

In [32]:
MINER_ADDRESS = bytes.fromhex('501ace0000000000000000000000000000000000')
MAIN_PERSISTANCE = './0x0B_naughty.sqlite'
ATTACK_PERSISTANCE = './0x0B_naughty_attack.sqlite'

## Remote Connection

In [33]:
REMOTE_NODES = ["ws://node.zimcoin.org:46030/"]
node = Node.start(MAIN_PERSISTANCE).proxy()
for remote in REMOTE_NODES:
    remote_connection(node, remote)


In [34]:
persistence = Persistence.start(MAIN_PERSISTANCE).proxy()
blocks = persistence.get_blocks().get()
for block in blocks:
    print(block.height, block.block_id.hex(), block.miner.hex(), block.difficulty)

0 001752953780439af7ad106e1aea94b1f855cad279868d65bbb590071385d9c2 bcf177e59d90d6c647918a0882c672be8b1fe289 1000
1 003551ec52bfbf104bbc8626e4a9cd5aa487f1e3206f78032a7cee9d3d798abc bcf177e59d90d6c647918a0882c672be8b1fe289 1000
2 00071decb5eaff91ebe8652d6a55d2d15c6a13d3ebffc69ceba793c0a8573258 bcf177e59d90d6c647918a0882c672be8b1fe289 1000
3 00210165f5fe47a4bbad0d2628258181ee220934e45dd2173496e9a67aa7d63a bcf177e59d90d6c647918a0882c672be8b1fe289 1000
4 002a8da3c2fff826a15a0c53adc126d80c387edac1315caf69f0a79554fbfe82 bcf177e59d90d6c647918a0882c672be8b1fe289 1000
5 00375b3349733c09fa132a771229ef7aa263975277c5326bcaa6c7b11e8d6795 bcf177e59d90d6c647918a0882c672be8b1fe289 1000
6 000d37e908dbc6ad3d72afc4b8679cad9aa06f862faf104b5f68329ba546726a bcf177e59d90d6c647918a0882c672be8b1fe289 1000
7 002ca379660f5d780fed5de13b34d1b0e6b785efaf1047e1c5b940ea26169df7 bcf177e59d90d6c647918a0882c672be8b1fe289 1000
8 003fbfa31f3a5bc2da482629e0eab2a4933bde97ba72c701dac4adc0ace3d946 bcf177e59d90d6c647918a0882c67

In [35]:
exclude_list = [
    bytes.fromhex('7a653da2cbffa16a1454ac3e6905edebce38e625'),
    bytes.fromhex('0000000000000000000000000000000000000000')
]

def get_target_blocks(start_height : int,
                      file_name : str) -> list:
    """
    Get a list of the blocks to attack.
    """
    attack_blocks = []
    blocks = Persistence.start(file_name).proxy().get_blocks().get()

    for block in tqdm(blocks, 'Getting target attack blocks'):
        if block.height >= start_height:
            attack_blocks.append(block)

    return attack_blocks

def attack_blocks(attack_blocks : list,
                  exclude_list : list,
                  attack_persistence : Persistence,
                  previous_block : bytes = None) -> None:
    """
    Attack the blocks and replace the miner address of miners in the 
    exclude list.
    """
    # create tme miner
    block_miner = ZimcoinMiner(
        platform_id=0,
        device_id=0,
        window_size=1e5)

    if previous_block is None:
        previous_block = attack_blocks[0].previous

    for block in tqdm(attack_blocks, 'Attacking blocks'):
        # determine the miner address to replace
        miner = block.miner if block.miner not in exclude_list else MINER_ADDRESS

        # mine the attacked block
        attacked_block = block_miner.mine(
            previous = previous_block,
            height = block.height,
            miner = miner,
            transactions = block.transactions,
            timestamp = block.timestamp,
            difficulty = block.difficulty)

        # save the attacked block
        previous_block = attacked_block.block_id
        print("Saving block", attacked_block.block_id.hex())
        attack_persistence.save_block(attacked_block)
        
def attack_range(start_height: int, previous_block : bytes) -> None:
    block_list = get_target_blocks(start_height, MAIN_PERSISTANCE)
    print(f'Attack Block Count : {len(block_list)}')

    # create a database for the output
    attack_file_name = ATTACK_PERSISTANCE
    # attack the blocks
    attack_blocks(
        attack_blocks=block_list, 
        exclude_list=exclude_list,
        attack_persistence=Persistence.start(attack_file_name).proxy(),
        previous_block=previous_block)

In [36]:
ATTACK_START = 3183 + 100
start_height = ATTACK_START
previous_block = None

In [37]:
# # get the blocks to attack blocks
# attack_range(start_height, previous_block)

In [38]:
# start_height = 4181
# previous_block = bytes.fromhex('00000010ff26f504a6b271cb6a5bf85f4a41946b5f4600cd4f8d8e34b3d37d85')
# attack_range(start_height, previous_block)

In [39]:
height = ATTACK_START
attacked_blocks = []
db = SqliteDict(ATTACK_PERSISTANCE, autocommit=True)
while True:
    block_dict = db.get(height)
    if block_dict is None:
        break
    attacked_blocks.append(dict_to_block(block_dict))
    height += 1

print(f'Attacked Block Count : {len(attacked_blocks)}')
print(f'First Block Height : {attacked_blocks[0].height}')
print(f'First block difficulty : {attacked_blocks[0].difficulty}')

last_block = attacked_blocks[-1]
print(f'Last block height : {last_block.height}')
print(f'Last block id : {last_block.block_id.hex()}')

# update the database
attack_range(last_block.height + 1, last_block.block_id)

Attacked Block Count : 1321
First Block Height : 3283
First block difficulty : 127430280
Last block height : 4603
Last block id : 00000000d27aa34d9951ab0cb7acb8120577dbefc288edcaf9afe848f2e88471


Getting target attack blocks:   0%|          | 0/4604 [00:00<?, ?it/s]

Attack Block Count : 0


Attacking blocks: 0it [00:00, ?it/s]

In [40]:
# get the existing list
existing_list = get_target_blocks(0, MAIN_PERSISTANCE)
existing_list = existing_list[:attacked_blocks[0].height]
print(f'Existing List last block height : {existing_list[-1].height}')
print(f'Attack List first block height : {attacked_blocks[0].height}')

# calculate the new state
new_state = BlockchainState([], dict(), 0)
for block in tqdm(existing_list, 'Building New State (existing)'):
    new_state.verify_and_apply_block(block)
for block in tqdm(attacked_blocks, 'Building New State (attack)'):
    new_state.verify_and_apply_block(block)

# mine a new block to add next to the attacked blocks
for index in range(1):
    new_block = mine_block(
        previous= attacked_blocks[-1].block_id,
        height=attacked_blocks[-1].height + 1,
        miner=MINER_ADDRESS,
        transactions=[],
        timestamp=int(time.time()),
        difficulty=new_state.calculate_difficulty())

    # add to the attacked blocks
    attacked_blocks.append(new_block)
    new_state.verify_and_apply_block(new_block)

# send the blocks to the nodes
existing_list.extend(attacked_blocks)
print(f'Existing List Block Count : {len(existing_list)}')

# create a new state
new_state = BlockchainState([], dict(), 0)
for block in tqdm(existing_list, 'Building New State'):
    new_state.verify_and_apply_block(block)
    print(block.height, block.block_id.hex())

for connection in node.connections.get():
    print(connection)
    print(connection.proxy().send_blocks(existing_list).get())

# new_state_summary.block_id
for connection in node.connections.get():
    block_id = existing_list[-1].block_id.hex()
    connection.proxy().send(json.dumps(dict(
        type='update_state',
        height=existing_list[-1].height,
        block_id=existing_list[-1].block_id,
        total_difficulty=new_state.total_difficulty
    ))).get()



Getting target attack blocks:   0%|          | 0/4604 [00:00<?, ?it/s]

Existing List last block height : 3282
Attack List first block height : 3283


Building New State (existing):   0%|          | 0/3283 [00:00<?, ?it/s]

Building New State (attack):   0%|          | 0/1321 [00:00<?, ?it/s]

*** Mining: miner=501ace0000000000000000000000000000000000, difficulty=689688720, device=Apple M1
**** Starting Iteration
**** Starting Iteration
**** Starting Iteration
**** Starting Iteration
*** Found block nonce: 784691970, block_id: 000000000ec008a976228a093da92d39cbc067e8de5cbc4c08ffaeeae6797de4
Existing List Block Count : 4605


Building New State:   0%|          | 0/4605 [00:00<?, ?it/s]

0 001752953780439af7ad106e1aea94b1f855cad279868d65bbb590071385d9c2
1 003551ec52bfbf104bbc8626e4a9cd5aa487f1e3206f78032a7cee9d3d798abc
2 00071decb5eaff91ebe8652d6a55d2d15c6a13d3ebffc69ceba793c0a8573258
3 00210165f5fe47a4bbad0d2628258181ee220934e45dd2173496e9a67aa7d63a
4 002a8da3c2fff826a15a0c53adc126d80c387edac1315caf69f0a79554fbfe82
5 00375b3349733c09fa132a771229ef7aa263975277c5326bcaa6c7b11e8d6795
6 000d37e908dbc6ad3d72afc4b8679cad9aa06f862faf104b5f68329ba546726a
7 002ca379660f5d780fed5de13b34d1b0e6b785efaf1047e1c5b940ea26169df7
8 003fbfa31f3a5bc2da482629e0eab2a4933bde97ba72c701dac4adc0ace3d946
9 001d9eeb153cb9a492b7c2e273b0e95a615ca05b9f5861b0e1acf6a1243be198
10 003e42ac27f436e578f1a896d0d28985632dc27348bc3319b7b280d6a90876f1
11 00000c7210bd8f48ac411d37688ec3558790ab12c369a137f48fec45d0a36369
12 00000016c20f6b7aab887f2b8ef56da45a5392289dda802db331dd52f60bd7de
13 00000098faaf47ac1c3096e733f148e776af90ba675863f27622d7474f8a2fc3
14 00000254900cbbf4e5c6a95623479d6f0f0bfa2ea5bfdfab5d02c42

In [44]:
for index in range(1):
    new_state = BlockchainState([], dict(), 0)
    for block in tqdm(attacked_blocks, 'Building New State'):
        new_state.verify_and_apply_block(block)

        for connection in node.connections.get():
            block_id = block.block_id.hex()
            connection.proxy().send_blocks([block]).get()
            connection.proxy().send(json.dumps(dict(
                type='update_state',
                height=block.height,
                block_id=block_id,
                total_difficulty=new_state.total_difficulty
            ))).get()

#time.sleep(10*60)

Building New State:   0%|          | 0/4605 [00:00<?, ?it/s]

Sending {"type": "blocks", "blocks": [{"previous": "0000000000000000000000000000000000000000000000000000000000000000", "height": 0, "miner": "bcf177e59d90d6c647918a0882c672be8b1fe289", "transactions": [], "timestamp": 1659431332, "difficulty": 1000, "block_id": "001752953780439af7ad106e1aea94b1f855cad279868d65bbb590071385d9c2", "nonce": 521}]}
Sending {"type": "update_state", "height": 0, "block_id": "001752953780439af7ad106e1aea94b1f855cad279868d65bbb590071385d9c2", "total_difficulty": 1000}
Sending {"type": "blocks", "blocks": [{"previous": "001752953780439af7ad106e1aea94b1f855cad279868d65bbb590071385d9c2", "height": 1, "miner": "bcf177e59d90d6c647918a0882c672be8b1fe289", "transactions": [], "timestamp": 1659431332, "difficulty": 1000, "block_id": "003551ec52bfbf104bbc8626e4a9cd5aa487f1e3206f78032a7cee9d3d798abc", "nonce": 1061}]}
Sending {"type": "update_state", "height": 1, "block_id": "003551ec52bfbf104bbc8626e4a9cd5aa487f1e3206f78032a7cee9d3d798abc", "total_difficulty": 2000}
Sen